In [2]:
import numpy as np
import pandas as pd
import ipdb
import pprinteps
eps = np.finfo(float).eps
from numpy import log2 as log
data = pd.read_csv("train.csv")
train,validate = np.split(data,[int(.8*len(data))])
train


ModuleNotFoundError: No module named 'pprinteps'

In [ ]:

def find_entropy(df):
    Class = "left"   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = float(df[Class].value_counts()[value])/len(df[Class])
        entropy += -fraction*np.log2(fraction+eps)
    return entropy

In [ ]:
def find_entropy_attribute(df,attribute):
  Class = "left"   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)

In [3]:
attributes = ["Work_accident", "promotion_last_5years", "sales", "salary"]    
my_dict = {}

In [4]:
def find_winner(df):
#     ipdb.set_trace()
    for key in attributes:
        if key == "left":
            continue
        my_dict[key] = find_entropy(df) - find_entropy_attribute(df,key)
        
    
    match = max(my_dict, key=my_dict.get)
#     print("Key")
#     print(match)
#     print("Value")
#     print(my_dict[match])
    print(my_dict[match])
    return match,my_dict[match]

In [5]:
def get_subtable(df, node,value):
#   print(value)
  return df[df[node] == value].reset_index(drop=True)


In [6]:

def buildTree(df): 
    Class = "left"   #To make the code generic, changing target variable class name

    #Here we build our decision tree

    #Get attribute with maximum information gain
    node,value = find_winner(df)
    
    if value == 0.0:
        #unique_labels = df['left'].unique()
        
        count_0 = len(df[df['left'] == 0])
        count_1 = len(df[df['left'] == 1])
        
        if count_0 > count_1:
            max = 0
        else:
            max = 1
        return max
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValues = df[node].unique()

    #Create an empty dictionary to create tree    
                       
    tree={}
    tree[node] = {}
    
    #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValues:
    #         ipdb.set_trace()
        subtable = get_subtable(df,node,value)
        unique_labels = subtable['left'].unique()                        

        if len(unique_labels)==1:   #Checking purity of subset
            tree[node][value] = unique_labels[0] 
            
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree
  
  


In [7]:
#pprint.pprint(buildTree(train))

In [8]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier
    
    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [9]:
tree = buildTree(train)

NameError: name 'train' is not defined

In [ ]:
for index, row in validate.iterrows():
    print(predict(row,tree))
    print(row["left"])
    print("********************************")

In [ ]:
def cal_TP_FP_TN_FN(tree):
    FP=0
    TP=0
    TN=0
    FN=0
    for index, row in validate.iterrows():
        predicted = predict(row,tree)
        actual = row["left"]
        if predicted == 0 :
            if actual == 0:
                TN+=1
            if actual == 1:
                FN+=1
        if predicted == 1:
            if actual == 0:
                FP+=1
            if actual == 1:
                TP+=1
    return TP,FP,TN,FN
                
            
        
        


In [ ]:
TP,FP,TN,FN = cal_TP_FP_TN_FN(tree)
print (TP)
print (FP)
print (TN)
print (FN)

In [ ]:
def calc_accuracy():
    total = TN+FP+TP+FN
    true = TN + TP
    accuracy = true / total
    print(accuracy)
    return accuracy

In [ ]:
accuracy = calc_accuracy()

In [ ]:
def calc_precision():
    positive = TP + FP
    precision = TP / positive
    print(precision)
    return precision

In [ ]:
precision = calc_precision()

In [ ]:
def calc_recall():
    actual_positive = TP + FN
    precision = TP / actual_positive
    print(precision)
    return precision

In [ ]:
recall = calc_recall()

In [ ]:
def calc_f1score():
    recall_inv = 1/recall;
    precision_inv = 1/precision
    den = recall_inv + precision_inv
    f1_score = 2 / den
    print(f1_score)
    return f1_score

In [ ]:
f1_score = calc_f1score()